# データ加工編　No.3

In [1]:
# 最初に実行すること
# 該当ノートブックに必要ないものも含まれているので注意

import os
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot
import seaborn as sns
import japanize_matplotlib

from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

from imblearn.under_sampling import RandomUnderSampler

In [2]:
warnings.filterwarnings('ignore')

# データの格納先が異なる場合は変更し、実行すること
df_customer = pd.read_csv('data/customer.csv')
df_category = pd.read_csv('data/category.csv')
df_product = pd.read_csv('data/product.csv')
df_receipt = pd.read_csv('data/receipt.csv')
df_store = pd.read_csv('data/store.csv')
df_geocode = pd.read_csv('data/geocode.csv')

## 課題 41〜60

---
41. レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算し、その結果を10件表示してください。

In [3]:
tmp_df = df_receipt.groupby('sales_ymd')['amount'].apply(lambda x: np.sum(x)).reset_index()

# 1行下にシフトする
tmp_df_diff = tmp_df.shift(1)
tmp_df_diff.columns = ['sales_ymd_shift', 'amount_shift']

# 横に結合する
tmp_df = pd.concat([tmp_df, tmp_df_diff], axis=1)
# 売上金額増減を計算
tmp_df['amount_diff'] = tmp_df['amount'] - tmp_df['amount_shift']

# 不要な列は削除（しなくてもOK）
tmp_df.head(10)

,sales_ymd,amount,sales_ymd_shift,amount_shift,amount_diff
0,20170101,33723,NaN,NaN,NaN
1,20170102,24165,20170101.0,33723.0,-9558.0
2,20170103,27503,20170102.0,24165.0,3338.0
3,20170104,36165,20170103.0,27503.0,8662.0
4,20170105,37830,20170104.0,36165.0,1665.0
5,20170106,32387,20170105.0,37830.0,-5443.0
6,20170107,23415,20170106.0,32387.0,-8972.0
7,20170108,24737,20170107.0,23415.0,1322.0
8,20170109,26718,20170108.0,24737.0,1981.0
9,20170110,20143,20170109.0,26718.0,-6575.0


---
42. レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合してください。そして結果を10件表示すること。

In [4]:
tmp_df = df_receipt.groupby('sales_ymd')['amount'].apply(lambda x: np.sum(x)).reset_index()

# ベースとなるDataFrameをコピー
df_lag = tmp_df.copy()


# 前回、前々回、3回前をそれぞれ追加
for i in range(1, 4):
    df_lag = pd.concat([df_lag, tmp_df.shift(i)], axis=1)
    columns = [f'lag_sales_ymd_{i}', f'lag_amount_{i}']
    # df_lagのcolumnsには、sales_ymdとamountが追加されるため、それを削除し新しいcolumnsを追加する
    df_lag.columns = list(df_lag.columns)[:-len(columns)] + columns

df_lag.dropna().astype(int).sort_values(['sales_ymd']).head(10)

,sales_ymd,amount,lag_sales_ymd_1,lag_amount_1,lag_sales_ymd_2,lag_amount_2,lag_sales_ymd_3,lag_amount_3
3,20170104,36165,20170103,27503,20170102,24165,20170101,33723
4,20170105,37830,20170104,36165,20170103,27503,20170102,24165
5,20170106,32387,20170105,37830,20170104,36165,20170103,27503
6,20170107,23415,20170106,32387,20170105,37830,20170104,36165
7,20170108,24737,20170107,23415,20170106,32387,20170105,37830
8,20170109,26718,20170108,24737,20170107,23415,20170106,32387
9,20170110,20143,20170109,26718,20170108,24737,20170107,23415
10,20170111,24287,20170110,20143,20170109,26718,20170108,24737
11,20170112,23526,20170111,24287,20170110,20143,20170109,26718
12,20170113,28004,20170112,23526,20170111,24287,20170110,20143


---
43. レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成してください。性別コードは0が男性、1が女性、9が不明を表すものとする。ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [5]:
tmp_df = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')

# math.floorは、指定された数値以下の最大の整数を表す数値を返す
tmp_df['era'] = tmp_df['age'].apply(lambda x: math.floor(x / 10) * 10)

tmp_df = pd.pivot_table(tmp_df, index='era', columns='gender_cd', values='amount', aggfunc='sum').reset_index()
tmp_df.columns = ['era', 'males_amount', 'females_amount', 'unknown_amount']
tmp_df

,era,males_amount,females_amount,unknown_amount
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


---
44.  上記（43.）で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換してください。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [6]:
#43と同じ
tmp_df = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')

tmp_df['era'] = tmp_df['age'].apply(lambda x: math.floor(x / 10) * 10)

tmp_df = pd.pivot_table(tmp_df, index='era', columns='gender_cd', values='amount', aggfunc='sum').reset_index()
tmp_df.columns = ['era', 'males', 'females', 'unknown']
tmp_df
# stack()で横持ちから縦持ちへ
tmp_df = tmp_df.set_index('era').stack().reset_index().replace({'males': '00', 'females': '01', 'unknown': '99'})
tmp_df.columns = ['era', 'gender_cd', 'amount']

tmp_df

,era,gender_cd,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


---
45. 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示してください。

In [7]:
tmp_birthday = pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')
pd.concat([df_customer['customer_id'], tmp_birthday], axis=1).head(10)

,customer_id,birth_day
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711


---
46. 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示してください。

In [8]:
tmp = pd.to_datetime(df_customer['application_date'])

pd.concat([df_customer['customer_id'], tmp], axis=1).head(10)

,customer_id,application_date
0,CS021313000114,1970-01-01 00:00:00.020150905
1,CS037613000071,1970-01-01 00:00:00.020150414
2,CS031415000172,1970-01-01 00:00:00.020150529
3,CS028811000001,1970-01-01 00:00:00.020160115
4,CS001215000145,1970-01-01 00:00:00.020170605
5,CS020401000016,1970-01-01 00:00:00.020150225
6,CS015414000103,1970-01-01 00:00:00.020150722
7,CS029403000008,1970-01-01 00:00:00.020150515
8,CS015804000004,1970-01-01 00:00:00.020150607
9,CS033513000180,1970-01-01 00:00:00.020150728


---
47. レシート明細データ（df_receipt）の売上エポック秒（sales_ymd）は数値型(int64)でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示してください。

In [9]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], pd.to_datetime(df_receipt['sales_ymd'].astype('str'))],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
48. レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示してください。

In [10]:
tmp = pd.to_datetime(df_receipt['sales_epoch'], unit='s').rename('sales_epoch_ymd')

pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], tmp], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
49. レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示してください。

In [11]:
tmp_sales_year = pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.year.rename('sales_epoch_year')

pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], tmp_sales_year], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch_year
0,112,1,2018
1,1132,2,2018
2,1102,1,2017
3,1132,1,2019
4,1102,2,2018
5,1112,1,2019
6,1102,2,2018
7,1102,1,2019
8,1112,2,2017
9,1102,1,2019


---
50. レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示してください。なお、「月」は0埋め2桁で取り出すこと。

In [12]:
tmp_sales_month = pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%m').rename('sales_epoch_month')

pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], tmp_sales_month], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch_month
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10


---
51.　レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示してください。なお、「日」は0埋め2桁で取り出すこと。

In [13]:
tmp_sales_day = pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%d').rename('sales_epoch_day')
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],tmp_sales_day], axis=1).head()

,receipt_no,receipt_sub_no,sales_epoch_day
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21


---
52.　レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示してください。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [14]:
tmp_df = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]\
        .groupby('customer_id')['amount'].apply(lambda x: np.sum(x)).reset_index()

tmp_df['over_2000'] = tmp_df['amount'].apply(lambda x: 1 if x > 2000 else 0)

tmp_df.head(10)

,customer_id,amount,over_2000
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
53. 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化してください。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [15]:
tmp_df = df_customer[['customer_id', 'postal_cd']].copy()
tmp_df['postal_flg'] = tmp_df['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)

# pd.merge(tmp_df, df_receipt, how='inner', on='customer_id').groupby('postal_flg').agg({'customer_id':'nunique'})
pd.merge(tmp_df, df_receipt, how='inner', on='customer_id').groupby('postal_flg')['customer_id'].nunique()

postal_flg
0    3906
1    4400
Name: customer_id, dtype: int64

---
54. : 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示してください。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [16]:
tmp_df = df_customer[['customer_id', 'address']].copy()

# 固定長で切り出す
# map()の場合、引数に辞書dictを指定すると要素の置換となる
tmp_df['prefecture_cd'] = tmp_df['address'].str[:3].map({'埼玉県': 11,
                                                         '千葉県': 12,
                                                         '東京都': 13,
                                                         '神奈川': 14})
tmp_df.head(10)

,customer_id,address,prefecture_cd
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


In [17]:
tmp_df = df_customer[['customer_id', 'address']].copy()

# 正規表現を使う
tmp_df['prefecture_cd'] = tmp_df['address'].str.extract(r'(^.*?[都道府県])')[0].map({'埼玉県': '11',
                                                                                    '千葉県':'12',
                                                                                    '東京都':'13',
                                                                                    '神奈川県':'14'})
tmp_df.head(10)

,customer_id,address,prefecture_cd
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
55. レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めてください。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示してください。カテゴリ値は順に1〜4とする。

- 最小値以上第1四分位未満 ・・・ 1を付与
- 第1四分位以上第2四分位未満 ・・・ 2を付与
- 第2四分位以上第3四分位未満 ・・・ 3を付与
- 第3四分位以上 ・・・ 4を付与

In [18]:
tmp_sum = df_receipt.groupby('customer_id')['amount'].sum()
# q1, q2, q3, q4 = np.percentile(tmp_sum, q=[i*25 for i in range(1, 5)])
q1, q2, q3, q4 = tmp_sum.quantile(q=[i*0.25 for i in range(1, 5)])

def change_category(x):
    if x < q1:
        return 1
    elif q1 <= x < q2:
        return 2
    elif q2 <= x < q3:
        return 3
    elif q3 <= x:
        return 4
tmp_df = df_receipt.groupby('customer_id')['amount'].sum().reset_index()
tmp_df['amount_category'] = tmp_df['amount'].apply(change_category)
tmp_df.head(10)

,customer_id,amount,amount_category
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


---
56. 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示してください。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [19]:
tmp_df = df_customer[['customer_id', 'birth_day']].copy()

tmp_df['era'] = df_customer['age'].apply(lambda x: min(math.floor(x / 10) * 10, 60))
# tmp_df['era'] = df_customer['age'].apply(lambda x: math.floor(x/10)*10)
# tmp_df['era'] = tmp_df['era'].apply(lambda x: 60 if x >= 60 else x)
tmp_df.head(10)

,customer_id,birth_day,era
0,CS021313000114,1981-04-29,30
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
5,CS020401000016,1974-09-15,40
6,CS015414000103,1977-08-09,40
7,CS029403000008,1973-08-17,40
8,CS015804000004,1931-05-02,60
9,CS033513000180,1962-07-11,50


---
57. 56.の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示してください。組み合わせを表すカテゴリの値は任意とする。

In [20]:
tmp_gender = df_customer[['customer_id', 'gender_cd']].copy()
df_era_gender = pd.merge(tmp_df, tmp_gender, on='customer_id', how='left')

# df_era_gender.dtypes
df_era_gender['era_gender'] = df_era_gender['era'].astype(str) + '_' + df_era_gender['gender_cd'].astype(str)
df_era_gender.head(10)

,customer_id,birth_day,era,gender_cd,era_gender
0,CS021313000114,1981-04-29,30,1,30_1
1,CS037613000071,1952-04-01,60,9,60_9
2,CS031415000172,1976-10-04,40,1,40_1
3,CS028811000001,1933-03-27,60,1,60_1
4,CS001215000145,1995-03-29,20,1,20_1
5,CS020401000016,1974-09-15,40,0,40_0
6,CS015414000103,1977-08-09,40,1,40_1
7,CS029403000008,1973-08-17,40,0,40_0
8,CS015804000004,1931-05-02,60,0,60_0
9,CS033513000180,1962-07-11,50,1,50_1


58. 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示してください。

In [21]:
tmp_df = pd.get_dummies(df_customer[['customer_id', 'gender_cd']], columns=['gender_cd'], drop_first=True)
tmp_df.head(10)

,customer_id,gender_cd_1,gender_cd_9
0,CS021313000114,1,0
1,CS037613000071,0,1
2,CS031415000172,1,0
3,CS028811000001,1,0
4,CS001215000145,1,0
5,CS020401000016,0,0
6,CS015414000103,1,0
7,CS029403000008,0,0
8,CS015804000004,0,0
9,CS033513000180,1,0


---
59. レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示してください。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

> TIPS:
> 標準化について確認してください。今回は、scikit-learnのpreprocessingに含まれるStandardScalerを使ってください。
> 「標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。」は、無視して良いとする。
> 余力があれば、pandas, Numpyを使って計算してみてください。この場合は、「標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。」を考慮する。


$$
z = \frac {x - \mu} {\sigma}   （ただし、\mu は平均、　\sigma　は標準偏差とする）
$$

In [22]:
# 標準化の式から算出
tmp_df = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
tmp_df = tmp_df.groupby('customer_id')['amount'].sum().reset_index()
tmp_amount_mean = tmp_df['amount'].mean()
# 不変分散の平方根を使う
tmp_amount_std = np.sqrt(tmp_df['amount'].var(ddof=1))

tmp_df['standard_amount'] = tmp_df['amount'].apply(lambda x: (x - tmp_amount_mean) / tmp_amount_std)
tmp_df.head(10)

,customer_id,amount,standard_amount
0,CS001113000004,1298,-0.459350
1,CS001114000005,626,-0.706348
2,CS001115000010,3044,0.182403
3,CS001205000004,1988,-0.205737
4,CS001205000006,3337,0.290096
5,CS001211000025,456,-0.768832
6,CS001212000027,448,-0.771773
7,CS001212000031,296,-0.827641
8,CS001212000046,228,-0.852635
9,CS001212000070,456,-0.768832


In [23]:
# scikit-learnのpreprocessingに含まれるStandardScalerを使うパターン
tmp_df = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
tmp_df = tmp_df.groupby('customer_id')['amount'].sum().reset_index()

scaler = preprocessing.StandardScaler()
scaler.fit(tmp_df[['amount']])

tmp_df['std_amount'] = scaler.transform(tmp_df[['amount']])

tmp_df.head(10)

,customer_id,amount,std_amount
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


---
60. レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示してください。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

> TIPS:
> 正規化について確認してください。今回は、scikit-learnのpreprocessingに含まれるMinMaxScalerを使ってください。
> 余力があれば、pandas, Numpyを使って計算してみてください。

In [24]:
# scikit-learnのpreprocessingに含まれるMinMaxScalerを使うパターン
tmp_df = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
tmp_df = tmp_df.groupby('customer_id')['amount'].sum().reset_index()

scaler = preprocessing.MinMaxScaler()
scaler.fit(tmp_df[['amount']])

tmp_df['std_amount'] = scaler.transform(tmp_df[['amount']])

tmp_df.head(10)

,customer_id,amount,std_amount
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771
